# Using VGG-16 as a base model for transfer learning

Now that I've trained a simple model using VGG-16 as a base, using only one Dense layer before output, I'd like to iterate on it to see how I can improve it. I'll be doing the same for InceptionV3 and comparing the two in order to make a final decision for my best model.

In [2]:
import matplotlib.pyplot as plt
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import models
from keras.layers import Dense, Flatten, Dropout
from keras.applications.vgg16 import VGG16
from pickle import dump

from functions import *

%load_ext autoreload
%autoreload 2

### Load data

In [3]:
# Image folder for training
train_dir = 'input_images/full_combined'

# Delete metadata files created by Mac OS
!find . -name ".DS_Store" -delete

In [4]:
# Make generators with image aug as before
train_datagen = ImageDataGenerator(rescale=1./255, 
                                    validation_split=0.1,
                                    horizontal_flip=True,
                                    rotation_range=20, 
                                    brightness_range=[0.5, 1.5], 
                                    zoom_range=.2)

train_gen = train_datagen.flow_from_directory(train_dir, subset='training', class_mode='binary')
val_gen = train_datagen.flow_from_directory(train_dir, subset='validation', class_mode='binary', shuffle=False)

Found 1548 images belonging to 2 classes.
Found 171 images belonging to 2 classes.


### Instantiate base model

I'll use the VGG-16 model without its original Dense layers for feature extraction. I'll then add my own Dense layers and sigmoid activation output.

In [ ]:
# Get base model
vgg_base_model = VGG16(weights='imagenet', 
                       include_top=False,
                       input_shape=(256, 256, 3))

# Freeze layers
for layer in vgg_base_model.layers:
    layer.trainable = False
    
# Check architecture
vgg_base_model.summary()